In [28]:
!pip install opencv-python
!pip install face_recognition opencv-python


In [29]:
import face_recognition
import numpy as np
from scipy.spatial import distance
import os
import urllib.request
import tarfile

In [30]:
def download_and_extract_lfw_dataset(url, extract_path):
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)
    tar_path = os.path.join(extract_path, 'lfw.tgz')
    if not os.path.exists(tar_path):
        print("Downloading LFW dataset...")
        urllib.request.urlretrieve(url, tar_path)
    if not os.path.exists(os.path.join(extract_path, 'lfw')):
        print("Extracting LFW dataset...")
        with tarfile.open(tar_path, "r:gz") as tar:
            tar.extractall(path=extract_path)
    print("LFW dataset is ready.")


lfw_url = "http://vis-www.cs.umass.edu/lfw/lfw.tgz"
lfw_extract_path = "./lfw_dataset"

download_and_extract_lfw_dataset(lfw_url, lfw_extract_path)

lfw_images_dir = os.path.join(lfw_extract_path, "lfw")

reference_image_path = os.path.join(lfw_images_dir, "George_W_Bush", "George_W_Bush_0001.jpg")


def extract_face_encodings(image_path):
    if not os.path.exists(image_path):
        raise ValueError(f"File does not exist: {image_path}")
    image = face_recognition.load_image_file(image_path)
    face_encodings = face_recognition.face_encodings(image)
    if len(face_encodings) == 0:
        raise ValueError(f"No face found in image: {image_path}")
    return face_encodings[0]


def compare_faces(encoding1, encoding2, tolerance=0.6, method='euclidean'):
    if method == 'euclidean':
        return distance.euclidean(encoding1, encoding2) < tolerance
    elif method == 'cosine':
        return distance.cosine(encoding1, encoding2) < tolerance
    else:
        raise ValueError("Unsupported comparison method")

#


LFW dataset is ready.


In [31]:
results = []
correct_euclidean = 0
correct_cosine = 0
incorrect_euclidean = 0
incorrect_cosine = 0


In [32]:
for test_image_path in test_image_paths[:1000]:  # Testowanie na 
    try:
        test_encoding = extract_face_encodings(test_image_path)
        euclidean_match = compare_faces(reference_encoding, test_encoding, method='euclidean')
        cosine_match = compare_faces(reference_encoding, test_encoding, method='cosine')

        if euclidean_match:
            correct_euclidean += 1
        else:
            incorrect_euclidean += 1

        if cosine_match:
            correct_cosine += 1
        else:
            incorrect_cosine += 1

        results.append({
            'image_path': test_image_path,
            'euclidean_match': euclidean_match,
            'cosine_match': cosine_match
        })
    except ValueError as e:
        results.append({
            'image_path': test_image_path,
            'error': str(e)
        })


In [33]:
for result in results:
    print(f"Image: {result['image_path']}")
    if 'error' in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Euclidean Match: {result['euclidean_match']}")
        print(f"Cosine Match: {result['cosine_match']}")
    print('-' * 30)

Image: ./lfw_dataset/lfw/Amy_Cotton/Amy_Cotton_0001.jpg
Euclidean Match: False
Cosine Match: True
------------------------------
Image: ./lfw_dataset/lfw/Fann_Wong/Fann_Wong_0001.jpg
Euclidean Match: False
Cosine Match: True
------------------------------
Image: ./lfw_dataset/lfw/Andrei_Mikhnevich/Andrei_Mikhnevich_0001.jpg
Euclidean Match: False
Cosine Match: True
------------------------------
Image: ./lfw_dataset/lfw/Andrei_Mikhnevich/Andrei_Mikhnevich_0002.jpg
Euclidean Match: False
Cosine Match: True
------------------------------
Image: ./lfw_dataset/lfw/Ryan_Newman/Ryan_Newman_0001.jpg
Euclidean Match: False
Cosine Match: True
------------------------------
Image: ./lfw_dataset/lfw/Gustavo_Terrazas/Gustavo_Terrazas_0001.jpg
Euclidean Match: False
Cosine Match: True
------------------------------
Image: ./lfw_dataset/lfw/Lorraine_Bracco/Lorraine_Bracco_0001.jpg
Euclidean Match: False
Cosine Match: True
------------------------------
Image: ./lfw_dataset/lfw/Valeri_Bure/Valeri_Bur

In [34]:

total_tests = correct_euclidean + incorrect_euclidean  # Liczba 
accuracy_euclidean = (correct_euclidean / total_tests) * 100 if total_tests > 0 else 0
accuracy_cosine = (correct_cosine / total_tests) * 100 if total_tests > 0 else 0

print(f"Total tests: {total_tests}")
# print(f"Euclidean - Correct: {correct_euclidean}, Incorrect: {incorrect_euclidean}, Accuracy: {accuracy_euclidean:.2f}%")
print(f"Cosine - Correct: {correct_cosine}, Incorrect: {incorrect_cosine}, Accuracy: {accuracy_cosine:.2f}%")

Total tests: 997
Cosine - Correct: 997, Incorrect: 0, Accuracy: 100.00%
